<a href="https://colab.research.google.com/github/DaverArenas/StockExchange_Predictions/blob/main/Random_Forest_Regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/jimpetoskey/random-forest-regressor/notebook

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Import necessary libraries
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
from matplotlib.pylab import rcParams
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy import stats
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Especializacion BigData EIA/TrabajoFinal/Overview_And_InitialPredictions/train_files'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/MyDrive/Especializacion BigData EIA/TrabajoFinal/Overview_And_InitialPredictions/train_files/financials.csv
/content/drive/MyDrive/Especializacion BigData EIA/TrabajoFinal/Overview_And_InitialPredictions/train_files/options.csv
/content/drive/MyDrive/Especializacion BigData EIA/TrabajoFinal/Overview_And_InitialPredictions/train_files/secondary_stock_prices.csv
/content/drive/MyDrive/Especializacion BigData EIA/TrabajoFinal/Overview_And_InitialPredictions/train_files/stock_prices.csv
/content/drive/MyDrive/Especializacion BigData EIA/TrabajoFinal/Overview_And_InitialPredictions/train_files/trades.csv


In [4]:
#Creamos un dataframe con los precios de las acciones que tenemos destinados para entrenamiento del modelo
prices = pd.read_csv("/content/drive/MyDrive/Especializacion BigData EIA/TrabajoFinal/Overview_And_InitialPredictions/train_files/stock_prices.csv")

In [5]:
prices.head(5)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [6]:
supplemental_prices = pd.read_csv("/content/drive/MyDrive/Especializacion BigData EIA/TrabajoFinal/Overview_And_InitialPredictions/supplemental_files/stock_prices.csv")

In [7]:
supplemental_prices.head(5)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20211206_1301,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False,-0.003263
1,20211206_1332,2021-12-06,1332,592.0,599.0,588.0,589.0,1360800,1.0,NaN,False,-0.008993
2,20211206_1333,2021-12-06,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,NaN,False,-0.009963
3,20211206_1375,2021-12-06,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,NaN,False,-0.015032
4,20211206_1376,2021-12-06,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,NaN,False,0.002867


In [8]:
prices_add = prices.append(supplemental_prices,ignore_index=True)

In [9]:
prices_add.head(5)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [10]:
prices_add['DateValue']=prices_add['Date'].str.replace('-','')

In [11]:
prices_add.head(5)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,DateValue
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,20170104
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324,20170104
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154,20170104
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053,20170104
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026,20170104


In [12]:
small_prices = prices_add[prices_add['DateValue']>'20200601']
small_prices=small_prices.drop(['DateValue'],axis=1)
small_prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009629 entries, 1592783 to 2602411
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   RowId             1009629 non-null  object 
 1   Date              1009629 non-null  object 
 2   SecuritiesCode    1009629 non-null  int64  
 3   Open              1004950 non-null  float64
 4   High              1004950 non-null  float64
 5   Low               1004950 non-null  float64
 6   Close             1004950 non-null  float64
 7   Volume            1009629 non-null  int64  
 8   AdjustmentFactor  1009629 non-null  float64
 9   ExpectedDividend  7876 non-null     float64
 10  SupervisionFlag   1009629 non-null  bool   
 11  Target            1009619 non-null  float64
dtypes: bool(1), float64(7), int64(2), object(2)
memory usage: 93.4+ MB


In [13]:
##ACCIONES NEGOCIADAS DESPUES DEL 2 DE JULIO DEL 2020 
small_prices.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
1592783,20200602_1301,2020-06-02,1301,2703.0,2737.0,2690.0,2730.0,11300,1.0,NaN,False,0.000727
1592784,20200602_1332,2020-06-02,1332,504.0,509.0,499.0,504.0,1865800,1.0,NaN,False,-0.003960
1592785,20200602_1333,2020-06-02,1333,2315.0,2342.0,2312.0,2327.0,156200,1.0,NaN,False,0.006858
1592786,20200602_1376,2020-06-02,1376,1380.0,1447.0,1380.0,1422.0,11200,1.0,NaN,False,-0.004892
1592787,20200602_1377,2020-06-02,1377,3640.0,3640.0,3600.0,3620.0,85000,1.0,NaN,False,-0.004121


##Feature Engineering Functions

In [14]:
def feature_train(train):
    
    #train = train.groupby("SecuritiesCode")
    
    # Add Lag Features
    lag_features = ["High", "Low", "Volume", "Close", "Open"]
    df_rolled_7d = train[lag_features].rolling(window=4, min_periods=0)
    df_mean_7d = df_rolled_7d.mean().shift(1).reset_index().astype(np.float32)
    df_mean_7d = df_mean_7d.drop('index', axis=1)
    df_mean_7d = df_mean_7d.fillna(0)
    df_mean_7d = df_mean_7d.round(2)
    df_mean_7d['High_lag_1'] = df_mean_7d['High']
    df_mean_7d['Open_lag_1'] = df_mean_7d['Open']
    df_mean_7d['Close_lag_1'] = df_mean_7d['Close']
    df_mean_7d['Volume_lag_1'] = df_mean_7d['Volume']
    df_mean_7d['Low_lag_1'] = df_mean_7d['Low']
    df_mean_7d = df_mean_7d.drop(["High", "Low", "Volume", "Close", "Open"], axis=1)
    train = train.reset_index(drop=True)
    train = pd.concat([train, df_mean_7d], axis=1)

    # Convert Date to float
    train['Date_Float'] = train['Date'].str.replace('-', '')
    train['Date_Float'] = train['Date_Float'].astype(float) 
    
    # Drop irrelevant columns for training
    train = train.drop(
                        ['RowId', 
                         'SupervisionFlag',
                         'AdjustmentFactor'], axis=1)
    
    # Bool to int for SupervisionFlag
    #train["SupervisionFlag"] = train["SupervisionFlag"].astype(int)
    
    # Backward, then forward fill missing values in cols
    cols = ['Target','Open', 'High', 'Low', 'Close']
    train.loc[:,cols] = train.loc[:,cols].bfill()
    train.loc[:,cols] = train.loc[:,cols].ffill()
    
    # Add Spread Features
    train['Daily_Spread'] = train['Close'] - train['Open']
    train['Daily_Max_Min'] = train['High'] - train['Low']
    train['1_Day_Spread_Close'] = train['Close'].diff()
    train['2_Day_Spread_Close'] = train['Close'].diff(periods=2)
    train['1_Day_Spread_Open'] = train['Open'].diff()
    train['2_Day_Spread_Open'] = train['Open'].diff(periods=2)
    train['1_Week_Spread'] = train['Close'].diff(periods=5)
    
    # Fill NaN's 
    train = train.fillna(0)
    
    # Add Return Features
    train['Return_Lag_1_Close'] = (train['Close'] - train['1_Day_Spread_Close'])/train['Close']
    train['Return_Lag_2_Close'] = (train['Close'] - train['2_Day_Spread_Close'])/train['Close']
    
    train['Return_Lag_1_Open'] = (train['Open'] - train['1_Day_Spread_Open'])/train['Open']
    train['Return_Lag_2_Open'] = (train['Open'] - train['2_Day_Spread_Open'])/train['Open'] 

    # Fill missing and inf/-inf values with 0
    train.replace([np.inf, -np.inf], 0, inplace=True)
    train = train.fillna(0)
        
    # Add rolling ratio of mean/std of forward 1 day return
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=5)
    train['ExPost_SR_Close'] = (train['Return_Lag_1_Close'].rolling(
        window=indexer, min_periods=1).mean())/(
        train['Return_Lag_1_Close'].std())
    
    train['ExPost_SR_2_Close'] = (train['Return_Lag_2_Close'].rolling(
        window=indexer, min_periods=1).mean())/(
        train['Return_Lag_2_Close'].std())
    
    train['ExPost_SR_Open'] = (train['Return_Lag_1_Open'].rolling(
        window=indexer, min_periods=1).mean())/(
        train['Return_Lag_1_Open'].std())
    
    train['ExPost_SR_2_Open'] = (train['Return_Lag_2_Open'].rolling(
        window=indexer, min_periods=1).mean())/(
        train['Return_Lag_2_Open'].std())
    
    # Fill missing and inf/-inf values with 0
    train.replace([np.inf, -np.inf], 0, inplace=True)
    train = train.fillna(0)
    
    # Fill missing values with 0
    #train = train.fillna(0)
    
    return train

##Min Max Scaler Function

In [15]:
def min_max(df):
    # MinMax Scale columns (-1, 1 scale)   
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled = scaler.fit_transform(df)
    train_cols = df.columns.values.tolist()
    trained = pd.DataFrame(data=scaled, columns=train_cols, index=df.index)
    return trained

##Build Training DataFrame Function

In [16]:
def build_train(df):
    # Groupby for feature engineering by securities code
    stock_list_df = df.groupby("SecuritiesCode").apply(feature_train)

    # reset index
    stock_list_df = stock_list_df.reset_index(drop=True)

    # Create Securities Code df
    sec_code_df = stock_list_df[['SecuritiesCode', 'Date']]
    
    # Drop Date column
    stock_list_df = stock_list_df.drop('Date', axis=1)

    # min max scale prices df
    stock_list_df = min_max(stock_list_df)

    # drop SecuritiesCode column before adding it back
    stock_list_df = stock_list_df.drop(['SecuritiesCode'], axis=1)

    # Concat dfs together
    stock_list_df = pd.concat([sec_code_df, stock_list_df], axis=1)

    # Sort df by date
    stock_list_df = stock_list_df.sort_values(by='Date')

    return stock_list_df

In [17]:
training_df = build_train(small_prices)

# Drop Date column from multi_model
single_model = training_df.drop('Date', axis=1)

In [18]:
# create Train X and y df's
X = single_model.drop('Target', axis=1)
y = single_model['Target']

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

In [20]:
# Instantiate and Fit Model
rf = RandomForestRegressor(random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train);

In [21]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [22]:
def funcion_ayuda(y_corr, y_pred):
  cm = confusion_matrix(y_corr, y_pred)
  sns.heatmap(cm, annot=True, fmt='.3f', square=True)
  acc = accuracy_score(y_corr, y_pred)
  plt.title(f'Accuracy: {round(acc,3)}')
  clasificacion_rep = pd.DataFrame(classification_report(y_corr, y_pred, output_dict=True))
  display(clasificacion_rep.round(2))

In [24]:
y_pred = rf.predict(X_test)

In [25]:
y_pred

array([-0.08121386, -0.09859916, -0.10356473, ..., -0.08020112,
       -0.08885268, -0.07811977])

In [26]:
len(y_pred)

201926

In [27]:
#Para verificar el error
from sklearn.metrics import mean_squared_error

In [28]:
J_mse = mean_squared_error(y_pred, y_test)
J_mse

0.0012132100776138155

In [29]:
rf.score(X_test, y_test)

0.27968934556822145

In [30]:
from sklearn.metrics import confusion_matrix

y = confusion_matrix(y_test, y_pred)

ValueError: ignored